In [ ]:
#data import 위에 놓으면 됩니다

#홍익선
#overspeed_cam.csv
#floating_pop.csv
#child_pedestrian.csv
#bump.csv
#parking.csv
#parking_cctv.csv

#조민주
#child_count
data = gpd.read_file("5.오산시_연령별_거주인구격자(유소년).geojson", encoding = "utf-8")
data.val[data.val.isna()] = 0
# data.to_json("child_count.geojson")

#child_rate_count
data = gpd.read_file("5.오산시_연령별_거주인구격자(유소년).geojson", encoding = "utf-8")
data.val[data.val.isna()] = 0
total_data = gpd.read_file("4.오산시_연령별_거주인구격자(총인구).geojson", encoding = "utf-8")
total_data.val[total_data.val.isna()] = 0
data["total"] = total_data.val
# data.to_json("child_rate_count.geojson")

#elem_kinder_count
data = pd.read_csv("10.오산시_학교위치정보.csv", encoding = "utf-8")
data = data[data["학교구분"] == "초등학교"]
data.rename(columns = {"학교위치_경도":"시설위치_경도", "학교위치_위도" : "시설위치_위도"}, inplace = True)
data = pd.read_csv("13.오산시_어린이집_유치원현황.csv", encoding = "utf-8")
school = pd.read_csv("10.오산시_학교위치정보.csv", encoding = "utf-8")
kinder = pd.read_csv("13.오산시_어린이집_유치원현황.csv", encoding = "utf-8")
elem = school[school["학교구분"] == "초등학교"]
data = kinder[["시설구분", "시설위치_경도", "시설위치_위도"]]
data.loc[:,"시설구분"] = "유치원과 어린이집"
elem = elem[["학교구분","학교위치_경도", "학교위치_위도"]].rename(columns = {"학교위치_경도":"시설위치_경도", "학교위치_위도" : "시설위치_위도","학교구분":"시설구분"})
data = pd.concat([data, elem], axis = 0)
data.index = range(len(data))
# data.to_csv("elem_kinder_count.csv")


#심재훈
#accident_count
data = pd.read_csv("2021_01_12_03_44_final.csv")
data.loc[:,'geometry'] = data.loc[:,['lat','lon']].apply(lambda x: (x.lat, x.lon),axis=1)
data.to_csv('accident_count.csv')

#chaos1
road_gdf = gpd.read_file('23.오산시_상세도로망_LV6.geojson')
road_to_geo = {}
for i in range(len(road_gdf)):
  road_to_geo[road_gdf.link_id[i]] = road_gdf.geometry[i]
chaos1_df = pd.read_csv('25.평일_전일_오산시_혼잡빈도강도_Level6.csv')
chaos1_df['geometry'] = chaos1_df.iloc[:,0].apply(lambda x: road_to_geo[str(x)[:-2]] if str(x)[:-2] in road_to_geo.keys() else False)
chaos1_df.columns = ['link_id','rank','length','name','sido','sigungo','eub','time_range','work/weekend','chaos1','geometry']
#to gdf
chaos1_gdf = gpd.GeoDataFrame(chaos1_df, crs='EPSG:4326')
# chaos1_gdf.to_csv('chaos1.csv')

#이재훈
#학원.csv
df = pd.read_csv("../data/original/30.오산시_학원_및_교습소_현황.csv")
# null 값 삭제
df = df[df.isna().sum(axis = 1) == 0] 
# 초등학교 학원이라 보기 힘든 것들 삭제
df = df[df["업종구분명"] != '평생직업교육학원']
df = df[~df["교습과정명"].apply(lambda x : "기사" in x) ]
def extract_noun(x):
    text_join = x
    filtered_content = re.sub(r'[^\d\s\w]', ' ', text_join)
    filtered_content = re.sub(r'[\d PABCDEFGHIJKLMNOPGRSTUVWXYZQ]', ' ', filtered_content)
    filtered_content = re.sub(r'[abcdefghijklmnopqrstuvwxyz*]', ' ', filtered_content)
    twitter = konlpy.tag.Twitter()
    twitter_morphs = twitter.pos(filtered_content)
    word = set([i[0] for i in twitter_morphs if i[1] == "Noun"])
    return word
df["교습과정명"] = df["교습과정명"].apply(extract_noun)
def check(x):
    if "초" in x or "초등" in x or "저학년" in x or "어린이" in x:
        return False
    try:
        for item in ["간호", "가구", "가죽", "기능사", "네일아트", "다이어트", "세무","실무","실비",
                     "정보처리","회계","회사", "고등", "중등", "중고", "중학생","고등학생", "중학교",
                     "고등학교", "기사" ,"성인","정보처리기사", "가죽공예","웹디자인" "화장품"]:
            if item in x:
                return True
        return False
    except:
        return False
df = df[~df["교습과정명"].apply(check)]
df = df.reset_index(drop = True)
# df.to_csv("학원.csv",index = False)

#횡단보도.json
df = gpd.read_file('17.오산시_횡단보도.geojson',encoding="utf-8")
df["geometry"] = df["geometry"].apply(lambda x: reproject(x, 'EPSG:4326', 'EPSG:26944'))
# df.to_file("횡단보도.json", driver="GeoJSON")

#인도.json
df = gpd.read_file('21.오산시_인도.geojson',encoding="utf-8")
df["geometry"] = df["geometry"].apply(lambda x: reproject(x, 'EPSG:4326', 'EPSG:26944'))
# df.to_file("인도.json", driver="GeoJSON")